In [7]:
import pandas as pd, numpy as np
import matplotlib, seaborn as sns
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7,3)
plt.rcParams['figure.constrained_layout.use'] = True

In [98]:
# load data and crop to earliest_all and latest_all
earliest_all = '2015-08-07'
latest_all = '2020-06-26'
mydateparser = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
mydateparser1 = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d')
btc_series = pd.read_excel('Data/BTC_closing.xlsx',squeeze=True, parse_dates=[0], index_col=0, date_parser=mydateparser)[earliest_all:latest_all]
eth_series = pd.read_excel('Data/ETH.xlsx',squeeze=True, parse_dates=[0], index_col=0, date_parser=mydateparser, usecols='A,E')[earliest_all:latest_all]

oil_WTI = pd.read_excel('Data/DCOILWTICO.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
oil_BRENT = pd.read_excel('Data/DCOILBRENTEU.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]

ex_JPN_USD = pd.read_excel('Data/DEXJPUS.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
ex_EUR_USD = 1 / pd.read_excel('Data/DEXUSEU.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
ex_GBP_USD = 1 / pd.read_excel('Data/DEXUSUK.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]

gold_10am = pd.read_excel('Data/GOLDAMGBD228NLBM10AM.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]
gold_03pm = pd.read_excel('Data/GOLDPMGBD228NLBM3PM.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]

dax = pd.read_excel('Data/SDAXI.xlsx', squeeze=True, parse_dates=[0], index_col=0, date_parser=mydateparser1, usecols=[0,5])[earliest_all:latest_all]
sp500 = pd.read_excel('Data/SP500.xls', squeeze=True, parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[earliest_all:latest_all]

In [99]:
print('BTC: ', len(btc_series), '\n', btc_series.index[0], '\n', btc_series.index[-1], '\n')
print('ETH: ', len(eth_series), '\n', eth_series.index[0], '\n', eth_series.index[-1], '\n')
print('OIL WTI: ', len(oil_WTI), '\n', oil_WTI.index[0], '\n', oil_WTI.index[-1], '\n')
print('OIL BRENT: ', len(oil_BRENT), '\n', oil_BRENT.index[0], '\n', oil_BRENT.index[-1], '\n')
print('JPN_USD: ', len(ex_JPN_USD), '\n', ex_JPN_USD.index[0], '\n', ex_JPN_USD.index[-1], '\n')
print('EUR_USD: ', len(ex_EUR_USD), '\n', ex_EUR_USD.index[0], '\n', ex_EUR_USD.index[-1], '\n')
print('GBP USD: ', len(ex_GBP_USD), '\n', ex_GBP_USD.index[0], '\n', ex_GBP_USD.index[-1], '\n')
print('GOLD 10am: ', len(gold_10am), '\n', gold_10am.index[0], '\n', gold_10am.index[-1], '\n')
print('GOLD 03pm: ', len(gold_03pm), '\n', gold_03pm.index[0], '\n', gold_03pm.index[-1], '\n')
print('DAX: ', len(dax), '\n', dax.index[0], '\n', dax.index[-1], '\n')
print('SP500: ', len(sp500), '\n', sp500.index[0], '\n', sp500.index[-1], '\n')

earliest_all = '2015-08-07'
latest_all = '2020-06-26'



BTC:  1786 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

ETH:  1786 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

OIL WTI:  1276 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

OIL BRENT:  1276 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

JPN_USD:  1276 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

EUR_USD:  1276 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

GBP USD:  1276 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

GOLD 10am:  1276 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

GOLD 03pm:  1276 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

DAX:  1239 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 

SP500:  1276 
 2015-08-07 00:00:00 
 2020-06-26 00:00:00 



In [106]:
len(dax)
# dax does not contain weekends and holidays, same for sp500
# traverse all data and pick dates only, where data for all is available


1239